# Association models with LDAK

Tutorial modified from the LDAK creator Doug Speed

For these practicals, we will use the les: `human.bed`, `human.bim` and
`human.fam` - 1000 Genomes Project SNP data for 424 humans, 3289 SNPs
(Chromosomes 21 & 22). `quant.pheno`, `binary.pheno`, `multi.pheno` and
`covar.covar` are phenotypes and covariates for these.

`hapmap.bed`, `hapmap.bim` and `hapmap.fam` - SNP data for 1184 humans,
1016 SNPs (from HapMap Project). `mice.bed`, `mice.bim`, `mice.fam` and
`mice.pheno` - SNP and phenotype data for 1940 mice, 2984 SNPs.

In [ ]:
wget https://www.dropbox.com/s/5vcfcree3xnvhew/data.zip

In [ ]:
unzip data.zip -d LDAKdata

In [ ]:
chmod -R 777 LDAKdata

# About LDAK

Designed in the style of PLINK. For example, all options start with two
dashes, if you want to provide data in binary `PLINK` format, you use
`--bfile`, to select a subset of SNPs you use `--extract`, provide
phenotypes with `--pheno`, perform linear regression with `--linear`,
etc.

LDKA has some similarity with `GCTA`: you merge kinship matrices using
`--add-grm`, estimate variance components using `--reml`, and kinship
matrices are stored in a compatible format. LDAK performs most features
of GCTA (not bivariate and some conditional analyses).

LDAK has a number of additional features (e.g., computing SNP
weightings, merging datasets, pro le scoring, thinning and clumping, as
well as GBAT for set-based association testing, SumHer for summary stats
heritability analysis, and MultiBLUP for prediction) LDAK can also
perform all the features of LDSC.

LDAK can accommodate almost any type of data. The formats are: Binary
PLINK (`--bfile`) - the most commonly used format, an [efficient way to
store hard genotypes](https://www.cog-genomics.org/plink/1.9/input#bed)
(all values are 0, 1, 2 or NA). [Gen / Chiamo / Oxford Stat
Format](http://www.stats.ox.ac.%20uk/~marchini/software/gwas/file_format.html)
(`--gen`) - rows are predictors, columns are SNPs, can set the number of
header rows and columns. Typically, used to provide state probabilities.
Can also store haplotype/phasing calls or dosage, etc. Files can be raw
text or gzipped.

SP Format (`--sp`) - Simple text matrix, with a row for each predictor
and a column for each individual. Value need not correspond to SNPs.
SPEED Format (`--speed`)- binary version of SP Format More info at
<http://www.ldak.org/file-formats/>.

Call LDAK by typing the name of the executable (same for PLINK, GCTA),
which will provide you informations about the program and which
arguments could be missing.

In [ ]:
ldak

LDAK options are provided in pairs. One main argument is required, which
is followed by the name of the output le. For example, suppose we want
to compute statistics, we would use the main argument `--calc-stats`.
The command might look like:

``` bash
ldak --calc-stats output --bfile human
```

This is asking LDAK to read the data stored in Binary PLINK format with
prefix `human`, then save the results to les with prefix `output`.
Option pairs can be provided in any order, so would be equivalent to use

``` bash
ldak --bfile human--calc-stats output
```

(The command above would compute MAF, call-rates and possibly info
scores)

## Single SNP association analysis

We wish to test each of the 3289 SNPs stored in the dataset human
individually for association with the continuous phenotype contained
within quant.pheno (using the model $Y = + alpha_j X_j$). For this we
use the main argument `--linear` Each main argument requires di erent
options. Documentation is provided at
[www.ldak.org](https://www.ldak.org), but typically the easiest way is
to run `LDAK` using just the main argument, then it will tell you what
options you require So here we type

In [ ]:
ldak --linear linear

First LDAK tells us we need to provide phenotypes, so we add that

In [ ]:
ldak --linear linear --pheno LDAKdata/quant.pheno

Now LDAK tells us we need to provide data les, so we add them

In [ ]:
ldak --linear linear --pheno LDAKdata/quant.pheno --bfile LDAKdata/human

> **Always pay attention at the output screen**
>
> It will tell you what options are required and why there are errors.
> It will also update you on progress (as will the file `.progress`) and
> tell you where the results are saved. For the linear regression, the
> main results are in `linear.assoc`. We will use `linear.summaries` for
> summary-statistic methods, `linear.pvalues` for clumping, while
> `linear.score` provides the polygenic risk score prediction model.

Some extra options we might wish to use are `--keep` (to use only a
subset of individuals), `--extract` (to use only a subset of predictors)
and `--covar` (to include covariates, such as age, sex, population
axes).

To achieve the same in PLINK, you would use

``` bash
plink --linear --out linear_plink --bfile LDAKdata/human --pheno LDAKdata/quant.pheno 
```

Note that by default PLINK ignores individuals without sex (add
`--allow-no-sex` to continue). If your phenotype is binary, you might
prefer logistic regression (use `--logistic` instead of `--linear`).

<img src="Images/R.png" alt="Bash" width="80"> Choose the R kernel

In [ ]:
assoc=read.table("linear.assoc",head=T)

In [ ]:
head(assoc,4)

In [ ]:
which.min(assoc[,7])

In [ ]:
assoc[71,]

In [ ]:
options(repr.plot.width=14, repr.plot.height=8)

par(mfrow=c(1,2)) 
plot(-log10(assoc[,7]), 
     col=assoc[,1], 
     pch=19,
     xlab="Chromosome", 
     ylab=expression(paste("-log"[10]," p-value",sep="")), 
     main="A Pretty Manhattan Plot",axes=F) 
marks=array(0,23) 
for(i in 1:22){
    a=which(assoc[,1]==i)
    marks[i+1]=marks[i]+length(a)
} 

marks2=(marks[-1]+marks[-23])/2 

axis(1,at=marks,rep("",23));

axis(1,at=marks2,lab=1:22,tick=F) 

axis(2);abline(h=-log10(5e-8),lwd=3,lty=2) 

ord=order(assoc[,7])

obsP=assoc[ord,7] 

N=nrow(assoc)

expP=1:N/(N+1) 

plot(-log10(expP), 
     -log10(obsP),
     pch=19,
     xlab="Expected-log10 P", 
     ylab="Observed-log10 P",
     main="A Less Pretty QQ-Plot") 

abline(a=0,b=1,col=2,lwd=3,lty=3) 

gif=qchisq(median(assoc[,7]),1,lower=F)/qchisq(.5,1) 

text(1.5,15,paste("Genomic Inflation Factor:",round(gif,2)),cex=1.3)

<img src="Images/bash.png" alt="Bash" width="80"> Choose the Bash kernel

To identify genome-wide associated SNPs we can use `awk`

In [ ]:
awk < linear.assoc '($7<5e-8){print$0}' 

To identify the independent loci, use the command `--thin-tops`; this
will find then prune all SNPs with p-values below a specified threshold.
`--thin-tops` requires lots of options, butagain, LDAK will walk you
through them…

In [ ]:
ldak --thin-tops top --bfile LDAKdata/human --pvalues linear.pvalues \
--cutoff 5e-8 --window-cm 1 --window-prune .2

In [ ]:
cat top.in

In [ ]:
awk '(NR==FNR){arr[$1];next}($2inarr){print$0}' top.in linear.assoc | head -n 10

## Preparing a GWAS

The steps are as follows:

1 - Remove poorly genotyped samples and SNPs 2 - Remove ethnic outliers
3 - Imputation 4 - Remove poorly imputed samples and SNPs 5 - Remove
related individuals

Steps 1, 2 & 3 are fairly constant. Steps 4 & 5 depend on the aim of the
GWAS; heritability analyses generally require much stricter quality
control than single-SNP analysis, and are designed for unrelated
individuals. One would also increase QC if studying a binary phenotype.
E.g., for a single-SNP analysis, I might use an info score threshold of
\>0.5 or \>0.8, but for heritability analysis require info \>0.99.

### Standard QC steps on the human data

We run some predefined thresholds on the human data

In [ ]:
plink --bfile LDAKdata/human \
    --geno 0.02 \
    --make-bed \
    --out Results/human1 --silent

plink --bfile Results/human1 \
    --mind 0.02 \
    --make-bed \
    --out Results/human2 --silent
    
plink --bfile Results/human2 \
    --maf 0.05 \
    --make-bed  \
    --out Results/human3 --silent

### Ethnic outliers and overlapping to a reference

Generally we wish to identify and exclude population outliers (the
exception is mixed-model association analysis). For this, we will
compute a kinship matrix (also referred to as a relatedness
matrix/allelic correlations).

The command for computing a kinship matrix is `--calc-kins-direct`. You
must use either `--weights` or `--ignore-weights YES` to specify SNP
weightings, and `--power` to specify how to standardize SNPs.

People normally use (in-sample) principal component analysis (PCA) in
this way:thin the SNPs, calculate a kinship matrix, perform PCA then
plot, and find outliers.

However, you cna also project the data onto population axes derived from
a diverse reference panel (e.g., HapMap):

1 - Find overlap of SNPs between dataset and reference panel 2 - Using
the reference panel, calculate a kinship matrix, perform PCA and compute
SNP loadings for the principal components 3 - Project the data onto
these principal components, then plot

#### Ethnic outliers

In [ ]:
#thin for LD 
ldak --bfile Results/human3 --thin prune \
    --window-prune .2 --window-cm 1 

This produces `prune.in` and `prune.out`

In [ ]:
#compute unweighted kinships 
ldak --calc-kins-direct prune --bfile Results/human3 \
    --ignore-weights YES --power -1 --extract prune.in \
    --kinship-raw YES 

The kinship matrix is stored with stem `prune`

In [ ]:
#calculate 5 principal components 
ldak --pca prune --grm prune --axes 5

Having computed the PCAs, we can plot and identify outliers

<img src="Images/R.png" alt="Bash" width="80"> Choose the R kernel

In [ ]:
datapca=read.table("prune.vect")
plot(datapca[,3:4],xlab="PC1",ylab="PC2")
abline(v=.07,col=2,lwd=3,lty=2)

### Projection from reference

<img src="Images/bash.png" alt="Bash" width="80"> Choose the bash kernel

In [ ]:
#get overlap between human and hapmap, then thin hapmap 
awk '(NR==FNR){arr[$2];next}($2 in arr){print $2}' \
    Results/human3.bim LDAKdata/hapmap.bim > overlap.txt 

ldak --thin thin --bfile LDAKdata/hapmap \
    --window-prune .2 --window-kb 1000 \
    --extract overlap.txt 
    

In [ ]:
#compute kinships, pcas and loadings 
ldak --calc-kins-direct hapmap --bfile LDAKdata/hapmap \
    --extract thin.in --ignore-weights YES --power -1 

In [ ]:
ldak --pca hapmap --grm hapmap --axes 20

In [ ]:
ldak --calc-pca-loads hapmap --bfile LDAKdata/hapmap \
    --grm hapmap --pcastem hapmap
    

In [ ]:
#project human onto these loadings 
ldak --calc-scores project --bfile Results/human3 \
    --scorefile hapmap.load --power 0

Plot the HapMap PCs, project onto these our data, then identify
outliers. Save a list of individuals to keep to `goodpop.keep`

<img src="Images/R.png" alt="Bash" width="80"> Choose the R kernel

In [ ]:
happca=read.table("hapmap.vect") 
proj=read.table("project.profile",head=T) 
par(mfrow=c(1,2)) 
plot(happca[,3:4],col="light grey",pch=19,xlab="Proj 1",ylab="Proj 2") 
text(c(-.04,.02,.02),c(0,.05,-.04),c("African","European","Asian")) 
plot(happca[,3:4],col="light grey",pch=19,xlab="Proj 1",ylab="Proj 2") 
points(proj[,c(5,7)],col=2,pch=19) 
abline(v=.005,col=2,lwd=3,lty=2) 
abline(h=.025,col=2,lwd=3,lty=2) 
keep=which(proj[,5]>.005&proj[,7]>.025) 
write.table(proj[keep,1:2],"goodpop.keep",row=F,col=F,quote=F)

Note that when using a proper number of individuals, the clusters will
be much tighter. Also, for convenience, here we used
[HapMap](ftp://ftp.ncbi.nlm.nih.gov/hapmap/genotypes/2009-01_%20phaseIII/plink_format),
but better to use 1000 Genomes Project (see
[www.ldak.org/reference-panel](www.ldak.org/reference-panel))

Below we plot both the PCA of the data itself and the PCA of the
projection on the reference dataset.

In [ ]:
datapca=read.table("prune.vect") 
happca=read.table("hapmap.vect") 
proj=read.table("project.profile",head=T) 
sites=rep(1,nrow(proj)) 
a=grep("AFR",proj[,2])
sites[a]=2 
a=grep("EAS",proj[,2])
sites[a]=3
a=grep("SAS",proj[,2])
sites[a]=4 
a=grep("EUR",proj[,2])
sites[a]=5
a=grep("AMR",proj[,2])
sites[a]=6 
a=grep("FIN",proj[,2])
sites[a]=7
par(mfrow=c(1,2)) 
plot(datapca[,3:4],col=sites,pch=19,xlab="PCA 1",ylab="PCA 2") 
plot(happca[,3:4],col="light grey",pch=19,xlab="Proj 1",ylab="Proj 2") 
points(proj[,c(5,7)],col=sites,pch=19)

We want to create a covariate le to use in subsequent regressions.
Generally include as covariates sex plus 10 principal components (5 from
in-sample PC, 5 from population projections, from the two plots above).
If you have imputed the data, repeat the PCA using that before using the
PCA as covariates.

In [ ]:
#can get sex from Column 5 of the fam file 
fam=read.table("LDAKdata/human.fam") 
#use match() to ensure samples are aligned 
m1=match(fam[,1],datapca[,1]) 
m2=match(fam[,1],proj[,1]) 
covar=cbind(fam[,c(1,2,5)],datapca[m1,3:7],proj[m2,c(5,7,9,11,13)]) 
write.table(covar,"covar.covar",row=F,col=F,quote=F)

## Remove relatedness

Remove Relatedness To estimate relatedness we can use the kinship matrix
computed when performing PCA (i.e., from thinned SNPs). Ideally you
would first remove population outliers. Notice that allelic correlation
kinships are distributed around expected values (e.g., 1/2 for
full-sibs, 0 for unrelateds ), and there are values above 1 and below 0.
(For proper sample sizes, spread will be smaller). The general
assumption is if we find identical individuals, these are accidental
duplicates, so we want to remove one of each pair. We can do this using
`filter`:

<img src="Images/bash.png" alt="Bash" width="80"> Choose the Bash kernel

In [ ]:
ldak --filter dups --grm prune --keep goodpop.keep --max-rel .8

(Of course, duplicates may be twins, so check clinical data if
possible).

If our aim is single-SNP analysis, we normally want to remove close
relatedness; a typical cut-off is 0.185, half-way between half-sibs and
cousins:

In [ ]:
ldak --filter close --grm prune --keep goodpop.keep --max-rel .375

(This is not necessary if performing mixed-model association analysis).

For heritability analyses, we generally require unrelated individuals,
so we lter based on estimated relatedness. As a threshold, you can use
an arbitrary value (say 0.025 or 0.05) or second cousins (1/32=0.03125),
but prefer to use the negative of the smallest value observed (the
default in LDAK)

In [ ]:
ldak --filter filter --grm prune --keep goodpop.keep

In this (unrealistic example), the minimum kinship is -0.176; using this
threshold results in removal of 9 individuals, so that 397 remain
(stored in `filter.keep`)

# SingleSNP analysis

Having performed quality control, now would be the correct time to
perform single-SNP analysis

In [ ]:
ldak --linear linear --pheno LDAKdata/quant.pheno --bfile Results/human3  --covar covar.covar

Look at the output message, which tells us the results are in the
`linear.assoc` file. The file looks like this

<img src="Images/R.png" alt="Bash" width="80"> Choose the R kernel

In [ ]:
results_log <- read.table("linear.assoc", head=TRUE)
head(results_log)

In [ ]:
# Setup to avoid long messages and plot on screen
options(warn=-1)
options(jupyter.plot_mimetypes = 'image/png')

# Load GWAS package qqman
suppressMessages(library("qqman"))

# Manhattan plot using --logistic results
results_log <- read.table("linear.assoc", head=TRUE)
#manhattan(results_log, main = "Manhattan plot: logistic", cex.axis=1.1)
manhattan(
  results_log,
  chr = "Chromosome",
  bp = "Basepair",
  p = "Wald_P",
  snp = "Predictor",
  col = c("pink", "blue"),
  suggestiveline = -log10(1e-7),
  genomewideline = 0,
  highlight = NULL,
  logp = TRUE,
  annotatePval = FALSE,
  annotateTop = TRUE
)

Which is the SNP with strongest association? We filter for smallest
SNPsas in the line on the plot above

In [ ]:
library(dplyr)
results_log %>% filter(Wald_P < .00001) %>% arrange(Wald_P)

# Conditional association analysis

We now wish to condition on the most strongly associated SNP
(21:15603999) Save this SNP to a file, then add the option
`--top-preds`:

<img src="Images/bash.png" alt="Bash" width="80"> Choose the Bash kernel

In [ ]:
echo "21:15603999" > top.snps ldak 

ldak --linear linear_cond \
    --pheno LDAKdata/quant.pheno \
    --bfile LDAKdata/human \
    --top-preds top.snps

This will include the top SNP as a (fixed-effect) covariate in the
regression (this is equivalent to making a covariate file containing the
SNP values, then using `--covar` instead of `--top-preds`) Previously,
we searched for independent loci by pruning; conditional analysis is a
more e ective (but slower) way to do this.

We see that when we condition on `21:15603999`, the two nearby SNPs are
no longer significant, because they were in high LD with it! You could
then also condition on the next top SNP (by adding it to the file
`top.snps`), and so on

<img src="Images/R.png" alt="Bash" width="80"> Choose the R kernel

In [ ]:
# Setup to avoid long messages and plot on screen
options(warn=-1)
options(jupyter.plot_mimetypes = 'image/png')

# Load GWAS package qqman
suppressMessages(library("qqman"))

# Manhattan plot using --logistic results
results_log <- read.table("linear_cond.assoc", head=TRUE)
#manhattan(results_log, main = "Manhattan plot: logistic", cex.axis=1.1)
manhattan(
  results_log,
  chr = "Chromosome",
  bp = "Basepair",
  p = "Wald_P",
  snp = "Predictor",
  col = c("pink", "blue"),
  suggestiveline = -log10(1e-7),
  genomewideline = 0,
  highlight = NULL,
  logp = TRUE,
  annotatePval = FALSE,
  annotateTop = TRUE
)

In [ ]:
library(dplyr)
results_log %>% filter(Wald_P < .00001) %>% arrange(Wald_P)

# Mixed model analysis

The standard linear model for testing SNP j is

$$Y = + \beta_j X_j$$

Mixed-model linear regression extends this to

$$Y = + \beta_j X_j + g where gi N(0, K \sigma^2_g)$$

and K is a kinship matrix The original idea was that including g allows
for long-range correlations due to relatedness and population structure.
But recently, it has been realised that **g** will also capture the
contribution of other causal variants, so in effect we are testing SNP j
using a multi-SNP model. Mixed-model association analysis is also
implemented in EMMA, FastLMM, GEMMA and GCTA.

The mice data contain large amounts of relatedness (all individualsare
derived from just 8 founders). Standard single-SNP analysis will give
near meaningless results. Look at the plots from the linear test:

<img src="Images/bash.png" alt="Bash" width="80"> Choose the Bash kernel

In [ ]:
ldak --linear stand --pheno LDAKdata/mice.pheno --bfile LDAKdata/mice

<img src="Images/bash.png" alt="R" width="80"> Choose the R kernel

In [ ]:
# Setup to avoid long messages and plot on screen
options(warn=-1)
options(jupyter.plot_mimetypes = 'image/png')

# Load GWAS package qqman
suppressMessages(library("qqman"))

# Manhattan plot using --logistic results
results_log <- read.table("stand.assoc", head=TRUE)
#manhattan(results_log, main = "Manhattan plot: logistic", cex.axis=1.1)
manhattan(
  results_log,
  chr = "Chromosome",
  bp = "Basepair",
  p = "Wald_P",
  snp = "Predictor",
  col = c("pink", "blue"),
  suggestiveline = -log10(1e-7),
  genomewideline = 0,
  highlight = NULL,
  logp = TRUE,
  annotatePval = FALSE,
  annotateTop = TRUE
)

In [ ]:
qq(results_log$Wald_P, main = "Q-Q plot of GWAS p-values (log) using linear model on mice")

However, results are dramatically improved by adding a kinship matrix

<img src="Images/bash.png" alt="Bash" width="80"> Choose the Bash kernel

In [ ]:
ldak --calc-kins-direct kinsm --bfile LDAKdata/mice\
    --ignore-weights YES --power -1 
ldak --linear mm --pheno LDAKdata/mice.pheno --bfile LDAKdata/mice --grm kinsm

<img src="Images/R.png" alt="Bash" width="80"> Choose the R kernel

In [ ]:
# Setup to avoid long messages and plot on screen
options(warn=-1)
options(jupyter.plot_mimetypes = 'image/png')

# Load GWAS package qqman
suppressMessages(library("qqman"))

# Manhattan plot using --logistic results
results_log <- read.table("mm.assoc", head=TRUE)
#manhattan(results_log, main = "Manhattan plot: logistic", cex.axis=1.1)
manhattan(
  results_log,
  chr = "Chromosome",
  bp = "Basepair",
  p = "Wald_P",
  snp = "Predictor",
  col = c("pink", "blue"),
  suggestiveline = -log10(1e-7),
  genomewideline = 0,
  highlight = NULL,
  logp = TRUE,
  annotatePval = FALSE,
  annotateTop = TRUE
)

In [ ]:
qq(results_log$Wald_P, main = "Q-Q plot of GWAS p-values (log) using linear model on mice")

# Estimate SNP heritability from data

To SNP heritability under the LDAK Model, there are three steps:

A - Calculate SNP weightings B - Calculate a weighted kinship matrix C -
Estimate the genetic and environmental variance via REML or Haseman
Elston or PCGC Regression (using only unrelated individuals). Those are
some of the possible techniques which are implemented also in other
softwares.

## SNP weights

The simple way to do this is using `--cut-weights` and
`--calc-weights-all`. This can take a lot of time and you need to be
patient.

<img src="Images/bash.png" alt="Bash" width="80"> Choose the bash kernel

In [ ]:
ldak --bfile LDAKdata/human --cut-weights sections \
    --keep goodpop.keep 

ldak --bfile LDAKdata/human --calc-weights-all sections \
    --keep goodpop.keep

## Weighted Kinship matrix

For this, we use `--calc-kins-direct`

In [ ]:
ldak --calc-kins-direct kins --bfile LDAKdata/human \
    --weights sections/weights.all\
    --power -.25 --kinship-raw YES

## Estimate variance components

First we consider the phenotype `quant.pheno` and use `REML`. Remember
to include `--keep` so that we only use unrelated individuals. The main
output file is `quant.reml`, but there are many additional files.

In [ ]:
ldak --reml quant --grm kins\
    --pheno LDAKdata/quant.pheno\
    --keep filter.keep

Note, large efect SNPs should be included as top predictors!

Heritability is estimated to be 0.73 +- 0.08

In [ ]:
cat quant.reml

# From multiple phenotypes

The file multi.pheno contains 10 traits. You can use the `--reml multi`
option for multiple phenotypes. We also need to decompose the kinship
matrix, which speeds up a lot the analysis computation. This is
especially useful when testing gene expression, where you can have more
than 20K phenotypes

In [ ]:
ldak --decompose kins --grm kins --keep filter.keep

ldak --reml multi --grm kins --eigen kins \
    --pheno LDAKdata/multi.pheno --keep filter.keep\
    --mpheno -1

In [ ]:
cat multi.10.reml